## fetching data for multiple coins

In [5]:
from re import S
import dotenv
import os
from requests import Session, RequestException
from pprint import pprint

dotenv.load_dotenv()

API_KEY = os.getenv("COINMARKETCAP_API_KEY")
API_ENDPOINT = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest"
COINS = "BTC,ETH"

s = Session()

response_api = s.get(
    url=API_ENDPOINT,
    headers={"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY},
    params={"symbol": COINS},
)

data = response_api.json()

pprint(data)

{'data': {'BTC': [{'circulating_supply': 19822878,
                   'cmc_rank': 1,
                   'date_added': '2010-07-13T00:00:00.000Z',
                   'id': 1,
                   'infinite_supply': False,
                   'is_active': 1,
                   'is_fiat': 0,
                   'last_updated': '2025-02-11T17:37:00.000Z',
                   'max_supply': 21000000,
                   'name': 'Bitcoin',
                   'num_market_pairs': 11931,
                   'platform': None,
                   'quote': {'USD': {'fully_diluted_market_cap': 2021253263258.67,
                                     'last_updated': '2025-02-11T17:37:00.000Z',
                                     'market_cap': 1907955087841.8364,
                                     'market_cap_dominance': 60.1797,
                                     'percent_change_1h': -0.24985292,
                                     'percent_change_24h': -1.04593482,
                                     '

### dataclass for api_connect handing over to producer

In [14]:
from dataclasses import dataclass


@dataclass
class CoinData:
    kafka_key: str
    kafka_value: dict


data = response_api.json().get("data", {})

coin_data = [
    CoinData(
        kafka_key=k,
        kafka_value=next(iter(data.get(k, {}))),
    )
    for k in data.keys()
]

pprint(coin_data)

[CoinData(kafka_key='BTC',
          kafka_value={'circulating_supply': 19822878,
                       'cmc_rank': 1,
                       'date_added': '2010-07-13T00:00:00.000Z',
                       'id': 1,
                       'infinite_supply': False,
                       'is_active': 1,
                       'is_fiat': 0,
                       'last_updated': '2025-02-11T17:37:00.000Z',
                       'max_supply': 21000000,
                       'name': 'Bitcoin',
                       'num_market_pairs': 11931,
                       'platform': None,
                       'quote': {'USD': {'fully_diluted_market_cap': 2021253263258.67,
                                         'last_updated': '2025-02-11T17:37:00.000Z',
                                         'market_cap': 1907955087841.8364,
                                         'market_cap_dominance': 60.1797,
                                         'percent_change_1h': -0.24985292,
               

### dataclass for remapping in consumer

In [15]:
@dataclass
class CoinQuote:
    symbol: str
    name: str
    last_updated: str
    price: float
    percent_change_1h: float


quotes = [
    CoinQuote(
        symbol=quote.kafka_value.get("symbol", ""),
        name=quote.kafka_value.get("name", ""),
        last_updated=quote.kafka_value.get("last_updated", ""),
        price=quote.kafka_value.get("quote", {}).get("USD", {}).get("price", ""),
        percent_change_1h=quote.kafka_value.get("quote", {})
        .get("USD", {})
        .get("percent_change_1h", ""),
    )
    for quote in coin_data
]

pprint(quotes)

[CoinQuote(symbol='BTC',
           name='Bitcoin',
           last_updated='2025-02-11T17:37:00.000Z',
           price=96250.15539327016,
           percent_change_1h=-0.24985292),
 CoinQuote(symbol='ETH',
           name='Ethereum',
           last_updated='2025-02-11T17:38:00.000Z',
           price=2640.25750847456,
           percent_change_1h=-0.27992486)]
